# Merging Tables with Different Join Types

---

## Left Join

**Import Libraries**

In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

**Load Datasets**

In [2]:
movies = pickle.load(
    open("D:/git_repositories/Datacamp-Joining_data_with_pandas/Datasets/movies.p", "rb")
)

In [5]:
movies.head()

,id,title,popularity,release_date
0,257,Oliver Twist,20.415572,2005-09-23
1,14290,Better Luck Tomorrow,3.877036,2002-01-12
2,38365,Grown Ups,38.864027,2010-06-24
3,9672,Infamous,3.680896,2006-11-16
4,12819,Alpha and Omega,12.300789,2010-09-17


In [7]:
# 2nd Dataset
financials = pickle.load(
    open("D:/git_repositories/Datacamp-Joining_data_with_pandas/Datasets/financials.p", "rb")
)

In [8]:
financials.head()

,id,budget,revenue
0,19995,237000000,2.787965e+09
1,285,300000000,9.610000e+08
2,206647,245000000,8.806746e+08
3,49026,250000000,1.084939e+09
4,49529,260000000,2.841391e+08


---

### Merging movies and financial tables as a left join

In [9]:
# Merge movies and financials with a left join
movies_financials = movies.merge(financials, on = 'id', how = 'left')

In [10]:
movies_financials.head()

,id,title,popularity,release_date,budget,revenue
0,257,Oliver Twist,20.415572,2005-09-23,50000000.0,42093706.0
1,14290,Better Luck Tomorrow,3.877036,2002-01-12,NaN,NaN
2,38365,Grown Ups,38.864027,2010-06-24,80000000.0,271430189.0
3,9672,Infamous,3.680896,2006-11-16,13000000.0,1151330.0
4,12819,Alpha and Omega,12.300789,2010-09-17,20000000.0,39300000.0


### Count the number of rows in the budget column that are missing

In [16]:
number_of_missing_fin = movies_financials['budget'].isnull().sum()
print("There are total " + str(number_of_missing_fin) + " missing values in budget column!")

There are total 1574 missing values in budget column!


---

### Enriching a dataset

In [18]:
# loading another dataset
taglines = pickle.load(
    open("D:/git_repositories/Datacamp-Joining_data_with_pandas/Datasets/taglines.p", "rb")
)

In [19]:
taglines.head()

,id,tagline
0,19995,Enter the World of Pandora.
1,285,"At the end of the world, the adventure begins."
2,206647,A Plan No One Escapes
3,49026,The Legend Ends
4,49529,"Lost in our world, found in another."


In [ ]:
# Merge action_movies to scifi_movies with right join
action_scifi = action_movies.merge(scifi_movies, on='movie_id', how='right',
                                   suffixes = ['_act', '_sci'])

# Print the first few rows of action_scifi to see the structure
print(action_scifi.head())

In [ ]:
# From action_scifi, select only the rows where the genre_act column is null
scifi_only = action_scifi[action_scifi['genre_act'].isnull()] 

In [ ]:
# Merge the movies and scifi_only tables with an inner join
movies_and_scifi_only = movies.merge(
    scifi_only,
    left_on = 'id',
    right_on = 'movie_id'
)